In [1]:
from StepParser import *
import time

In [2]:
# https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1
import psutil
import ray
import sys

num_cpus = psutil.cpu_count(logical=False)

ray.shutdown()
ray.init(num_cpus = num_cpus)

2020-06-19 15:45:58,272	INFO resource_spec.py:212 -- Starting Ray with 3.27 GiB memory available for workers and up to 1.65 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-19 15:45:58,632	INFO services.py:1170 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '172.29.121.196',
 'raylet_ip_address': '172.29.121.196',
 'redis_address': '172.29.121.196:15882',
 'object_store_address': '/tmp/ray/session_2020-06-19_15-45-58_266540_13612/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-19_15-45-58_266540_13612/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-06-19_15-45-58_266540_13612'}

In [3]:
@ray.remote
def parse_definition2(line):
    return parse_definition(line)
    
def pool_parse_file1(filename):
    result = []
    block_size = 1024*8
    remainder = ''
    
    with open(filename) as fp:
        while True:
            chunk = fp.read(block_size)
            if not chunk:
                break
            v = remainder + chunk
            lines = []
            
            for line in v.splitlines():
                if line.rstrip().endswith(';'):
                    remainder = ''
                    lines.append(line)
                else:
                    remainder = line
            result_ids = []
            for line in lines:
                result_ids.append(parse_definition2.remote(line))
            results = ray.get(result_ids)
            
            result.extend(results)
                
    return result

# print(pool_parse_file1('ar.ifc'))

In [4]:
import time
from StepParser import *
import yaml

start = time.time()
a = parse_file('bolt.ifc')

print('parsed in ', time.time() - start)

start = time.time()
a = pool_parse_file1('bolt.ifc')

print('parsed parallel in ', time.time() - start)

#with open('test3.yaml','w') as file:
#    for aa in a:
#       docs = yaml.dump(aa, file)


parsed in  0.3702890872955322
parsed parallel in  2.606706380844116


In [6]:
d = pd.DataFrame(a)

d


,0
0,ISO-10303-21;
1,HEADER;
2,"FILE_DESCRIPTION((''), '2;1');"
3,FILE_NAME('D:\KOSTA\BIM_structures_Kosta\Task_...
4,FILE_SCHEMA(('IFC2X3'));
...,...
1051,[1045] IFCRELAGGREGATES
1052,[1046] IFCRELCONTAINEDINSPATIALSTRUCTURE
1053,[1047] IFCRELDEFINESBYPROPERTIES
1054,ENDSEC;
